# Caltrans Data Extraction

The goal of this project is to extract relevant data from text files, previously converted from PDF files. Since the text files are quite structured, the decision is to use regex to do this.

## Setup

Install the following packages if you don't have them yet:

In [ ]:
# pip install pandas numpy tqdm ipykernel notebook python-dotenv openpyxl

In [ ]:
from experiment import *

import pyperclip

%reload_ext autoreload
%autoreload 2

# Note: to print DataFrame fully use:
# pd.set_option('display.max_rows', None)  # to set globally, or use: 
# with pd.option_context('display.max_rows', None, 
#                        'display.max_columns', None, 
#                        'display.width', None, 
#                        'display.max_colwidth', None):
#   display(df)

## Rename the mislabeled contracts

There are some files that were mistakenly labeled, so here we rename them:

In [ ]:
d = {'07-0W0404..pdf_12652.txt': '07-0W0404.pdf_12652.txt',
     '110427R0.pdf_3052.txt': '01-0A0804.pdf_3052.txt',
     '110427R0.pdf_2981.txt': '01-0A0804.pdf_2981.txt',
     '120712R0.pdf_4254.txt': '11-270804.pdf_4254.txt',
     '08-1N0304..pdf_12819.txt': '08-1N0304.pdf_12819.txt',
     '120928R0.pdf_4565.txt': '04-4S1204.pdf_4565.txt',
     '121016R0.pdf_4699.txt': '04-4S0304.pdf_4699.txt',
     '130220R0.pdf_4863.txt': '03-3E6204.pdf_4863.txt',
     '120717R0.pdf_4252.txt': '05-1A9704.pdf_4252.txt',
     '08-1G2804..pdf_12877.txt': '08-1G2804.pdf_12877.txt'
}

for key, value in d.items():
    try:
        path1 = RAW_DATA_PATH_LINEPRINTER / key
        path2 = RAW_DATA_PATH_LINEPRINTER / value
        path1.rename(path2)

        path1 = RAW_DATA_PATH_TABLE / key
        path2 = RAW_DATA_PATH_TABLE / value
        path1.rename(path2)
    except FileNotFoundError:
        print(f'File not found: {key}')
        continue


## Classify contracts into types

There are 3 main types of contract (type 3 is merged contracts that need to be parsed first):

In [ ]:
# sort_contracts()

In [ ]:
contract_types, _ = get_contract_types()
contract_types

## Single Contract Analysis

Let's look at the single contract:

In [ ]:
c = Contract('04-290834_619')

To copy file contents to clipboard:

In [ ]:
# pyperclip.copy(c.file_contents)

Now we extract:

In [ ]:
c.extract()

There are four attributes of the contract that gets extracted: info, bids, subcontractors, and items:

In [ ]:
c.items.df

We can also write all the information into excel file:

In [ ]:
c.write_to_disk()

In [ ]:
# filepath = RAW_DATA_PATH.parent / 'sample' / '01-0A3804.pdf_2724.txt'
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0A3804.pdf_4353.txt'
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0A0904.pdf_2724.txt'
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0A1204.pdf_11468.txt'
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0F4304.pdf_12346.txt'  # issue # 11
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0K6104.pdf_12731.txt'  # issue # 9
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0K4604.pdf_12040.txt'  # issue # 1
# filename = '01-0H3204.pdf_9871.txt'  # issue # 5
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0A0404.pdf_10165.txt'  # different format
# filepath = RAW_DATA_PATH_LINEPRINTER / '04-4G6404.pdf_7310.txt'


# Several contract analysis

We now run extraction on a small sample of contracts:

In [ ]:
df_contract_types, _ = get_contract_types()
df_contract_types

In [ ]:
filepaths = get_contract_filepaths(ContractType.TYPE2)
ex = Experiment(filepaths)
ex.run()

# Process all contracts

In [ ]:
filepaths = get_contract_filepaths(contract_type=ContractType.TYPE1)

In [ ]:
filepaths = get_contract_filepaths(contract_type=ContractType.TYPE2)

In [ ]:
filepaths = get_contract_filepaths(contract_type=ContractType.TYPE3)

Let's exclude some known errors:

In [ ]:
exclude_type1 = ['12-0K0234_11520', '12-0R2704_10602', '06-0L8404_3005', '07-0W0404_12652', '07-338004_11638']
exclude_type2 = ['09-237704_3534']
exclude = exclude_type1 + exclude_type2
filepaths = [x for x in filepaths if x.stem not in exclude]
len(filepaths)

In [ ]:
c  = Contract('type1/11-2N0034_10706')
pyperclip.copy(c.file_contents)


In [ ]:
c.info.extract()

In [ ]:
ex = Experiment('type3/03-2F5704_11589')
ex.run()

In [ ]:
ex = Experiment(filepaths)
ex.run()